In [49]:
#load packages
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
from pandas import Series, DataFrame
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

#misc libraries
import random
import time


#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)
# データインポートライブラリ
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
import warnings
import requests
import zipfile
import io
import re as re
import string

# データ加工・処理・分析ライブラリ
import scipy as sp
import sweetviz as sv


# 可視化ライブラリ
from pandas_profiling import ProfileReport
%matplotlib inline


#Model
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc
%precision 3


SEED = 42


Python version: 3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]
pandas version: 1.3.2
matplotlib version: 3.4.3
NumPy version: 1.20.1
SciPy version: 1.7.1
IPython version: 7.26.0
scikit-learn version: 0.24.2
-------------------------


In [50]:
def concat_df(train_data, test_data, sort=True, drop=True):
    """
    overview
    >return a  concatenated df of training and test data
    needed
    >pandas as pd
    input
    >train_data{DataFrame}:train_data
    >test_data{DataFrame}:test_data
    >sort{bool}:True or False
    """
    return pd.concat([train_data, test_data], sort=sort).reset_index(drop=drop)


def divide_df(all_data, target, len=890):
    """
    overview
    > Returns divided dfs of training and test set
    needed
    >
    input
    > all_data{DataFrame}:Train+Test
    > target{str} :Target columns
    > len(int):len of train_data
    
    """
    return all_data.loc[:len], all_data.loc[len:].drop([target], axis=1)


In [51]:
df_train = pd.read_csv("./df/EDA_05_train.csv")
df_test = pd.read_csv("./df/EDA_05_test.csv")
Target = 'Survived'
Final_target = 'Perished'
df_test.shape

(418, 44)

In [52]:
df_all = concat_df(df_train, df_test)
drop_cols = ['Deck', 'Embarked', 'Family', 'Family_Size', 'Family_Size_Grouped', Target,
             'Name', 'Parch', 'PassengerId', 'Pclass', 'Sex', 'SibSp', 'Ticket', 'Title',
             'Ticket_Survival_Rate', 'Family_Survival_Rate', 'Ticket_Survival_Rate_NA', 'Family_Survival_Rate_NA']

df_all.drop(columns=drop_cols, inplace=True)

df_all.head()

,Age,Deck_1,Deck_2,Deck_3,Deck_4,Embarked_1,Embarked_2,Embarked_3,Family_Size_Grouped_1,Family_Size_Grouped_2,...,Pclass_3,Sex_1,Sex_2,Survival_Rate,Survival_Rate_NA,Ticket_Frequency,Title_1,Title_2,Title_3,Title_4
0,2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.383838,0.0,1,0.0,0.0,0.0,1.0
1,7,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.000000,1.0,2,0.0,0.0,1.0,0.0
2,4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,1.0,0.0,0.383838,0.0,1,0.0,0.0,1.0,0.0
3,7,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.383838,0.0,2,0.0,0.0,1.0,0.0
4,7,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.383838,0.0,1,0.0,0.0,0.0,1.0


In [53]:
X_train = StandardScaler().fit_transform(df_train.drop(columns=drop_cols))
y_train = df_train[Target].values
X_test = StandardScaler().fit_transform(df_test.drop(columns=drop_cols))

print('X_train shape: {}'.format(X_train.shape))
print('y_train shape: {}'.format(y_train.shape))
print('X_test shape: {}'.format(X_test.shape))

X_train shape: (891, 26)
y_train shape: (891,)
X_test shape: (418, 26)


In [54]:
#Machine Learning Algorithm (MLA) Selection and Initialization
MLA = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    linear_model.LogisticRegressionCV(),
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Nearest Neighbor
    neighbors.KNeighborsClassifier(),
    
    #SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    svm.LinearSVC(),
    
    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    XGBClassifier()    
    ]



#split dataset in cross-validation with this splitter class: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit
#note: this is an alternative to train_test_split
cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .6, random_state = 0 ) # run model 10x with 60/30 split intentionally leaving out 10%

#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy Mean', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD' ,'MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns)

#create table to compare MLA predictions
MLA_predict = y_train

#index through MLA and save performance to table
row_index = 0
for alg in MLA:

    #set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
    cv_results = model_selection.cross_validate(alg, X_train, y_train, cv  = cv_split,return_train_score=True)

    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()   
    #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
    MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3   #let's know the worst that can happen!
    

    #save MLA predictions - see section 6 for usage
    alg.fit(X_train, y_train)
    
    row_index+=1

    
#print and sort table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
MLA_compare
#MLA_predict


,MLA Name,MLA Parameters,MLA Train Accuracy Mean,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
19,LinearDiscriminantAnalysis,"{'covariance_estimator': None, 'n_components':...",0.855993,0.853358,0.047241,0.007823
8,RidgeClassifierCV,"{'alphas': array([ 0.1, 1. , 10. ]), 'class_w...",0.855805,0.852985,0.052552,0.009171
3,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...",0.915543,0.852239,0.060531,0.106151
6,LogisticRegressionCV,"{'Cs': 10, 'class_weight': None, 'cv': None, '...",0.861049,0.850373,0.066121,0.439657
16,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...",0.857491,0.848134,0.058392,0.031194
0,AdaBoostClassifier,"{'algorithm': 'SAMME.R', 'base_estimator': Non...",0.865356,0.843284,0.065272,0.103137
14,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.880524,0.842537,0.058338,0.059613
21,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...",0.954494,0.840672,0.039434,0.117535
5,GaussianProcessClassifier,"{'copy_X_train': True, 'kernel': None, 'max_it...",0.916854,0.838806,0.055475,0.22308
15,NuSVC,"{'break_ties': False, 'cache_size': 200, 'clas...",0.854494,0.837313,0.045388,0.076105


In [55]:
print(MLA_compare.loc[21,:])
print(MLA_compare.loc[21,"MLA Parameters"])

MLA Name                                                       XGBClassifier
MLA Parameters             {'objective': 'binary:logistic', 'use_label_en...
MLA Train Accuracy Mean                                             0.954494
MLA Test Accuracy Mean                                              0.840672
MLA Test Accuracy 3*STD                                             0.039434
MLA Time                                                            0.117535
Name: 21, dtype: object
{'objective': 'binary:logistic', 'use_label_encoder': False, 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'gamma': None, 'gpu_id': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': Non

In [56]:
Pick_up_Model = [
    discriminant_analysis.LinearDiscriminantAnalysis(),
    XGBClassifier(),
    ensemble.GradientBoostingClassifier(),    
    ]
i =0
cv_split = model_selection.ShuffleSplit(
        n_splits=10, test_size=.3, train_size=.6, random_state=0)

Model1 = Pick_up_Model[0]
cv_results = model_selection.cross_validate(
    Model1, X_train, y_train, cv=cv_split, return_train_score=True)
Model1.fit(X_train,y_train) 
Model1_pred = Model1.predict_proba(X_test)

Model2 = Pick_up_Model[1]
cv_results = model_selection.cross_validate(
    Model2, X_train, y_train, cv=cv_split, return_train_score=True)
Model2.fit(X_train, y_train)
Model2_pred = Model2.predict_proba(X_test)

Model3 = Pick_up_Model[2]
cv_results = model_selection.cross_validate(
    Model3, X_train, y_train, cv=cv_split, return_train_score=True)
Model3.fit(X_train, y_train)
Model3_pred = Model3.predict_proba(X_test)


In [1]:
pred_proba = (Model3_pred)/1
type(pred_proba)
predict = pred_proba.argmax(axis=1)
predict

NameError: name 'Model3_pred' is not defined

In [79]:
submission_ex = pd.read_csv("gender_submission.csv")
submission_df = pd.DataFrame(columns=['PassengerId', Target])
submission_df['PassengerId'] = df_test['PassengerId']
submission_df.reset_index(drop=True, inplace=True)
submission_df[Target] = predict
submission_df.to_csv('sub/06_submissions_kaggle.csv', header=True, index=False)
print("Submission Format:{} ".format(submission_ex.shape))
print("My Sumbisison Format:{}".format(submission_df.shape))
submission_df


Submission Format:(418, 2) 
My Sumbisison Format:(418, 2)


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [80]:

submission_ex = pd.read_csv("gender_submission.csv")
submission_df = pd.DataFrame(columns=['PassengerId', Target])
submission_df['PassengerId'] = df_test['PassengerId']
submission_df[Target] = predict
submission_df[Final_target] = 0
submission_df.loc[submission_df[Target] == 0,Final_target] = 1
submission_df.loc[submission_df[Target] == 1, Final_target] = 0
submission_df.drop(Target, axis=1, inplace=True)

submission_df.reset_index(drop=True, inplace=True)
submission_df.to_csv('sub/06_submissions.csv', header=True, index=False)
print("Submission Format:{} ".format(submission_ex.shape))
print("My Sumbisison Format:{}".format(submission_df.shape))
submission_df


Submission Format:(418, 2) 
My Sumbisison Format:(418, 2)


,PassengerId,Perished
0,892,1
1,893,1
2,894,1
3,895,1
4,896,0
...,...,...
413,1305,1
414,1306,0
415,1307,1
416,1308,1
